모듈 API에서는 모듈내에 정의한 함수 또는 클래스 만큼이나 발생시킬 예외도 API의 일부분으로서 중요하다.

파이썬 언어와 표준 라이브러리에는 이미 예외 계층 구조가내장돼 있다. 

예를 들어 직접 만든 모듈의 함수에 잘못된 파라미터가 전달되면 ValueError 예외를 던질 수도 있다.

In [ ]:
# my_module.py

def determine_weight(volume, density):
    if density <= 0:
        raise ValueError('밀도는 0보다 커야합니다.')

In [5]:
# 하지만 API의 경우 새로운 Exception을 정의하고 모듈이 발생시키는 다른 모든 예가 이 최상위 예외를 상속하게 만듬으로써 API에서 발생하는 예외의 계층 구조를 만들 수 있다.

# my_module.py

class Error(Exception):
    """이 모듈에서 발생할 모든 예외의 상위 클래스"""
    
class InvaildDensityError(Error):
    """밀도 값이 잘못된 경우"""
    
class InvalidVolumeError(Error):
    """부피 값이 잘못된 경우"""
    
def determine_weight(volume, density):
    if density < 0:
        raise InvaildDensityError('밀도는 0보다 커야합니다.')
    if volume < 0:
        raise InvalidVolumeError('부피는 0보다 커야합니다.')
    if volume == 0:
        density / volume
    


In [2]:
# 어떤 모듈안에 최상위 예외가 있으면 API 사용자들이 이 모듈에서 발생한 모든 오류를 더 쉽게 잡아낼 수 있다.
import logging
try:
    weight = determine_weight(1, -1)
except Error:
    logging.exception('예상지 못한 오류')
    
    
    
# logging.exception 함수가 잡아낸 예외의 전체 스택 트레이스를 출력하기 때문에 더 쉽게 이 상황을 디버깅 할 수 있다.

ERROR:root:예상지 못한 오류
Traceback (most recent call last):
  File "/tmp/ipykernel_660985/2859667870.py", line 4, in <module>
    weight = determine_weight(1, -1)
  File "/tmp/ipykernel_660985/2879308889.py", line 16, in determine_weight
    raise InvaildDensityError('밀도는 0보다 커야합니다.')
InvaildDensityError: 밀도는 0보다 커야합니다.


이렇게 try/except 문을 사용하면 우리 모듈에서 발생한 예외가 모듈을 호출하는 코드로부터 아주 멀리 절단돼 프로그램이 깨지는 상황을 방지 할 수 있다.

이런 식으로 최상위 예외가 있으면 우리가 제공하는 API로 부터 호출하는 코드를 보호할 수 있다.

이렇게 보호하면 세 가지 유용한 효과가 나타난다.

1. 최상위 예외가 있으면 API를 호출하는 사용자가 API를 잘못 사용한 경우를 더 쉽게 이해할 수 있다는 점이다.

호출자가 API를 제대로 사용한다면 API에서 의도적으로 발생시킨 여러 예외를 잡아내야만 한다.

사용자가 이런 예외를 잡아내지 않으면, 우리가 만든 모듈의 최상위 예외르 잡아내는 방어적인 except 블록까지 예외가 전달된다. 이 블록은

APi 사용자의 주의를 환기 시키고 사용자가 깜빡한 여러 타입을 제대로 처리할 기회를 제공한다

In [6]:
try:
    weight = determine_weight(-1, 1)
except InvaildDensityError:
    weight = 0
except Error:
    logging.exception('호출코드에 버그가 있음')

ERROR:root:호출코드에 버그가 있음
Traceback (most recent call last):
  File "/tmp/ipykernel_660985/3701826053.py", line 2, in <module>
    weight = determine_weight(-1, 1)
  File "/tmp/ipykernel_660985/2380107141.py", line 18, in determine_weight
    raise InvalidVolumeError('부피는 0보다 커야합니다.')
InvalidVolumeError: 부피는 0보다 커야합니다.


2. API 모듈의 코드의 버그를 발견할 떄 도움이 된다는 뜻이다.

우리가 작성한 모듈 코드는 의도적으로 모듈 내에서 정의한 예외 계층에 속하는 예외만 발생시킬 수 있다.

이 경우 다른 타입의 예외가 발생한다면, 이 예외는 우리가 의도하지 않은 것이다. 즉, 우리가 구현한 API 코드에 버그가 있다는 뜻이다.

모듈의 최상위 예외를 잡아내는 try/except 문이 모듈의 버그로부터 API 소비자들을 보호하지는 못한다. 그러므로 호출하는 쪽에서 파이썬 기반 Except 클래스를 잡아내는 다른 except 블록을 추가해야한다.

이렇게 두 가지 except 문을 사용하면 API 소비자가 API 모듈에 수정해야할 버그가 있는 경우를 쉽게 감지할 수 있다.



In [7]:
try:
    weight = determine_weight(0, 1)
except InvaildDensityError:
    weight = 0
except Error:
    logging.exception('호출 코드에 버그가 있음')
except Exception:
    logging.exception('API 코드에 버그가 있음')
    raise # 예외를 호출자 쪽으로 다시 발생시킴

ERROR:root:API 코드에 버그가 있음
Traceback (most recent call last):
  File "/tmp/ipykernel_660985/3277768524.py", line 2, in <module>
    weight = determine_weight(0, 1)
  File "/tmp/ipykernel_660985/2380107141.py", line 20, in determine_weight
    density / volume
ZeroDivisionError: division by zero


ZeroDivisionError: division by zero

세 번째 효과는 미래의 API를 보호해준다는 점이다. API 확장을 할때 특정 상황에서 더 구체적인 예외를 제공한다면



In [ ]:
class NegativeDensityError(InvaildDensityError):
    """밀도가 음수인 경우"""
    ...
    
def determine_weight(volume, density):
    if density < 0:
        raise NegativeDensityError('밀도는 0보다 커야합니다.')
    if volume < 0:
        raise InvalidVolumeError('부피는 0보다 커야합니다.')
    if volume == 0:
        density / volume

In [9]:
# 모듈을 호출하는 코드는 변경하지 않아도 예전과 똑같이 잘 작동한다.

try:
    #
    weight = determine_weight(1, -1)
except NegativeDensityError as exc:
    raise ValueError('밀도로 음수가 아닌 값을 제공해야 합니다') from exc
except InvaildDensityError:
    weight = 0
except Error:
    logging.exception('호출 코드에 버그가 있음')
except Exception:
    logging.exception('API 코드에 버그가 있음!')
    raise # 예외를 호출자쪽으로 다시 발생시킴

NameError: name 'NegativeDensityError' is not defined

최상위 예외 바로 아래 폭넓은 예외 상황을 표현하는 다양한 오류를 제공하면 미래의 코드변경에 대한 보호를 더 강화할 수 있다.

In [ ]:
class Error(Exception):
    """이 모듈에서 발생할 모든 예외의 상위 클래스."""


class InvalidDensityError(Error):
    """밀도 값이 잘못된 경우."""


class InvalidVolumeError(Error):
    """부피 값이 잘못된 경우."""


class NegativeDensityError(InvalidDensityError):
    """밀도가 음수인 경우."""

ㅖ